# Example 2: Analyzing LAMMPS thermodynamic data

Author: [Richard Berger](mailto:richard.berger@outlook.com)

This tutorial assumes you've completed the [first example](simple.ipynb) and understand the basics of running LAMMPS through Python. In this tutorial we will build on top of that example and look at how to extract thermodynamic data produced by LAMMPS into Python and visualize it. Let's first start by recreating our simple melt example:

In [ ]:
%load_ext lammps.ipython

In [ ]:
from lammps import lammps
L = lammps()
L.cmd.auto_flush = True

def init_melt_system(L):
    # 3d Lennard-Jones melt
    L.cmd.clear()
    L.cmd.units("lj")
    L.cmd.atom_style("atomic")
    
    L.cmd.lattice("fcc", 0.8442)
    L.cmd.region("box", "block", 0, 4, 0, 4, 0, 4)
    L.cmd.create_box(1, "box")
    L.cmd.create_atoms(1, "box")
    L.cmd.mass(1, 1.0)
    
    L.cmd.velocity("all", "create", 1.44, 87287, "loop geom")
    
    L.cmd.pair_style("lj/cut", 2.5)
    L.cmd.pair_coeff(1, 1, 1.0, 1.0, 2.5)
    
    L.cmd.neighbor(0.3, "bin")
    L.cmd.neigh_modify("delay", 0, "every", 20, "check no")
    
    L.cmd.fix("1", "all", "nve")
    
    L.cmd.thermo(50)

Here we take advantage of the fact that we can write regular Python functions to organize our LAMMPS simulation. This allows us to clear and initialize a new system by calling the `init_melt_system()` function. With this we can now go ahead an run this simulation for 100 steps.

In [ ]:
init_melt_system(L)
L.cmd.run(100)

## Extracting thermodynamic data

Looking at the above output we see that LAMMPS prints out thermodynamic data for steps 0, 50 and 100.

```
   Step          Temp          E_pair         E_mol          TotEng         Press     
         0   1.44          -6.7733681      0             -4.6218056     -5.0244179    
        50   0.70303849    -5.6796164      0             -4.629178       0.50453907   
       100   0.72628044    -5.7150774      0             -4.6299123      0.29765862
```

We could parse the text output and extract the necessary information, but this has proven to be error-prone and clunky, especially in cases where other output gets interleaved with thermo output lines. Instead, we can make use of the Python integration within LAMMPS to execute arbitrary Python code during time steps using `fix python/invoke`. We can extract the thermodynamic data directly using the LAMMPS Python interface and process it in any way we want.

For this we first define the data structure we want to use to store the data. For each column of the thermodynamic data we want to store a list of values for each time step. Let's use a Python `dict` with the following structure:

```python
{'Step': [0, 50, 100, ...], 'Temp': [...], 'E_pair': [...], 'E_mol': [...], 'TotEng': [...], 'Press': [...]}
```

To start, let's define an empty `dict` and call it `current_run`. As the simulation progresses, we append new data into this dictionary:

In [ ]:
current_run = {}

Next, let's define a function that should be executed every time step a thermodynamic output line would be written. This function takes a `lammps` class instance and through it can access LAMMPS state and data. We can use the [`last_thermo()`](https://docs.lammps.org/Python_module.html#lammps.lammps.last_thermo) function of the `lammps` class to get the latest thermodynamic data as a dictionary. This data is all we need to populate our `current_run` data structure.

In [ ]:
def append_thermo_data(lmp):
  for k, v in lmp.last_thermo().items():
    current_run.setdefault(k, []).append(v)

With these two pieces in place, it is now time to tell LAMMPS about how we want to call this function.

First, let's suppress any LAMMPS output via `%%capture_lammps_output` and reinitialize our system with `init_melt_system()` so our system is back in its initial state and the time step is back to 0.

Next, we add a new fix `python/invoke` that should execute every 50 time steps, the same as our `thermo 50` command above. At the end of every 50 time steps (including the first one), it should call the `append_thermo_data` function we just defined. Notice we can just pass the function as parameter. Finally, we tell LAMMPS to run for 250 steps.

In [ ]:
%%capture_lammps_output
init_melt_system(L)
L.cmd.fix("myfix", "all", "python/invoke", 50, "end_of_step", append_thermo_data)
L.cmd.run(250)

Let's inspect our `current_run` dictionary after the run has completed:

In [ ]:
current_run

As we can see, the time steps 0, 50, 100, 150, and 200 were added to dictionary. However, the last time step 250 is still missing. For this we need to manually add a final call to our `append_thermo_data()` helper function.

In [ ]:
append_thermo_data(L)

With this our `current_run` dictionary now has all the data of the completed run:

In [ ]:
current_run

## Plotting thermodynamic data with matplotlib

Now that we have our data available as Python variables, we can easily use other libraries for visualization.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

plt.xlabel('time step')
plt.ylabel('Total Energy')
plt.plot(current_run['Step'], current_run['TotEng'])

## Using Pandas library

Since we can call any Python code from LAMMPS, the above example can also be rewritten using the Pandas library:

In [ ]:
%%capture_lammps_output
import pandas as pd

current_run = pd.DataFrame()

def append_thermo_data(lmp):
    global current_run
    current_time_step = pd.DataFrame.from_records([lmp.last_thermo()])
    current_run = pd.concat([current_run, current_time_step], ignore_index=True)

init_melt_system(L)
L.cmd.fix("myfix", "all", "python/invoke", 50, "end_of_step", append_thermo_data)
L.cmd.run(250)
append_thermo_data(L)
current_run

In [ ]:
current_run.plot(x='Step', y='TotEng')

# Conclusion

The Python interface gives you a powerful way of invoking and extracting simulation data while the simulation is running. Next we'll look at how to extract information about the atoms in your system.

<div style="text-align:right"><a href="atoms.ipynb">Next</a>